In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyserini==0.9.4.0

     |████████████████████████████████| 60.4MB 46kB/s 
     |████████████████████████████████| 1.1MB 50.5MB/s 


In [ ]:
!pip3 install transformers
!pip3 install unidecode

     |████████████████████████████████| 1.3MB 11.2MB/s 
     |████████████████████████████████| 2.9MB 58.3MB/s 
     |████████████████████████████████| 890kB 62.6MB/s 
     |████████████████████████████████| 1.1MB 44.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b58327c74e0b77c7422d65318023da24ef4e79461744b7c12ac5bc1c42bbf38b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 245kB 13.2MB/s 


In [ ]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import join
import unidecode
import re
import logging
from tqdm.notebook import tnrange
import glob
import json

#For ploting results
import matplotlib.pyplot as plt

# DL Libraries
from transformers import BertModel, AdamW, BertTokenizer,RobertaTokenizer, BertConfig, get_linear_schedule_with_warmup,RobertaModel, RobertaForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,TensorDataset)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from statistics import mean 
#NLTK Libraries
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=d19386f658d7d9556d92a482948114644c589d1ae78b6809ac3b4e59d7cca99d
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.5 GB  |     Proc size: 545.2 MB
GPU RAM Free: 15069MB | Used: 10MB | Util   0% | Total     15079MB


In [ ]:
from pyserini.search import SimpleSearcher
import pandas as pd
from os.path import join
import numpy as np

searcher = SimpleSearcher('/content/drive/My Drive/phase2_experiments/anserini/pyserini/indexes/lucene-index-qasc')

searcher.set_bm25(1.2, 0.75)

hits = searcher.search("the work horses of the sponge have tiny hairs that trap the particles")

# Print the first 10 hits:
for i in range (0,len(hits)):
    print(f'{hits[i].docid:15} {hits[i].score:.5f}')

16995292        23.37420
16995289        20.71960
16995285        16.20840
16995291        15.71050
12223761        15.31550
16995288        15.31550
16995290        14.54090
16995286        11.43330
7710175         11.31500
16996147        10.71430


In [ ]:
for i in range (0,len(hits)):
  doc = searcher.doc(str(hits[i].docid))
  print(doc.raw())

the work horses of the sponge have tiny hairs that trap the particles
sponges have tiny hairs that trap particles
Choanocytes have tiny hairs that trap the particles.
flagellum are tiny hairs that trap particles
Collar cells have tiny hairs that trap the particles.
The interior chambers have tiny hairs that trap the particles
sponges trap particles
Collar cells have tiny hairs that trap very tiny pieces of matter
Tiny particles trapped in vapor in a gas stream.
sponges have specialized work horses that produce constant flow of water to the sponge.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("device: {} n_gpu: {}".format(device, n_gpu))

device: cuda n_gpu: 1


# Data Loader

In [ ]:
def create_dataloader(tokenizer, df):
    input_ids= list()
    attention_masks= list()

    special_sentences_1 = [sentence for i, sentence in enumerate(df.question)]
    special_sentences_2 = [" [SEP] " + str(sentence) for i, sentence in enumerate(df.answer)]
    special_sentences = [i + j for i, j in zip(special_sentences_1, special_sentences_2)]

    for sentence in special_sentences:
      encoded_text = tokenizer.encode_plus(sentence, max_length=512, add_special_tokens=True, return_token_type_ids=False, 
                                       padding='max_length', return_attention_mask=True, truncation=True)
      input_ids.append(encoded_text['input_ids'])
      attention_masks.append(encoded_text['attention_mask'])

    inputs = torch.tensor(input_ids).to(device)
    masks = torch.tensor(attention_masks).to(device)
    # gold_labels = torch.tensor(df.label.tolist()).to(device)
  
    data = TensorDataset(inputs, masks)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=1)

    return dataloader

## Evaluation

In [ ]:
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

In [ ]:
def recall_at_k(actual, predicted, k):
  count=0
  predicted= predicted[0:k]
  for a in actual:
    if a in predicted:
      count+=1
  return count/len(actual)

In [ ]:
def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

In [ ]:
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

#Fetch sentences from corpus

In [ ]:
def get_candidate_passages(query):

  candidate_passages=[]
  hits = searcher.search(query, k=50)
  # return the first top 10 hits:
  for hit in hits:
    doc = searcher.doc(str(hit.docid))
    candidate_passages.append(doc.raw().replace('"', ''))
  
  return candidate_passages

#Model

In [ ]:
#Class for Regression
class Regressor(nn.Module):

  def __init__(self, model_path):
    super(Regressor, self).__init__()
    self.bert = RobertaModel.from_pretrained(model_path)
    self.out = nn.Linear(self.bert.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    output, pooler_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    score= self.out(pooler_out)
    return score

In [ ]:
#classification model

# model_path= '/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/sia_experiment/model_QASCrobertabase_40ktraindataset_10epochs'
model_path= '/content/drive/My Drive/phase2_experiments/SIA_EXPERIMENT/model_QASCrobertabase_fulltraindataset_10epochs'

#Load Model
model= Regressor(model_path)
lr_weights= torch.load(join(model_path, 'model_state.bin'))
model.out.load_state_dict(lr_weights)
model.to(device)


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Load Dev set


In [ ]:
#Loadibg the QASC preprocessd Data
#dev_questions=pd.read_csv('/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/input_csv/dev_question.csv')
data_df=pd.read_csv('/content/drive/My Drive/phase2_experiments/Data/dev_qfc.csv')

#evaluation

# Experiment 1 only combined fact

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  # if(index>50):
  #   break
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      score = model(ip_ids, attention_mask=masks)
      pos_prob.append(score.cpu().detach().numpy())
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  org_ans.append(row.exact_question)

  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.exact_question:
      contains_combined_fact+=1
  # if(index==1000):
  #   break
  print(index)
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
print("QASC DEV experiment 1 with considering only combined facts")
print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_combined_fact",contains_combined_fact)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Experiment 2 all the three facts

In [ ]:
data_df=pd.read_csv('/content/drive/My Drive/phase2_experiments/Data/dev_facts.csv')

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_all_three=0
contains12=0
contains1c=0
contains2c=0
for index, row in data_df.iterrows():
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      score = model(ip_ids, attention_mask=masks)
      pos_prob.append(score.cpu().detach().numpy())
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  org_ans.append(row.fact_1)
  org_ans.append(row.fact_2)
  org_ans.append(row.combined_fact)

  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)):
    contains_all_three+=1
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact not in top_k_ans)):
    contains12+=1
  if ((row.fact_1 in top_k_ans) and (row.combined_fact in top_k_ans) and (row.fact_2 not in top_k_ans)):
    contains1c+=1
  if ((row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)and (row.fact_1 not in top_k_ans)):
    contains2c+=1

  
  print(index)
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
print("QASC DEV experiment 2 with considering all the three facts")
print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_all_three",contains_combined_fact)
print("contains_fact1_and_fact2",contains12)
print("contains_fact1_and_factc",contains1c)
print("contains_fact2_and_factc",contains2c)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Experiment 3 only fact1




In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact1=0
for index, row in data_df.iterrows():
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      score = model(ip_ids, attention_mask=masks)
      pos_prob.append(score.cpu().detach().numpy())
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  org_ans.append(row.fact_1)

  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_1:
      contains_fact1+=1
  # if(index==1000):
  #   break
  print(index)
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
print("QASC DEV experiment 3 with considering only fact1")
print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_fact1",contains_fact1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Experiment 4 only combined fact2

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact2=0
for index, row in data_df.iterrows():
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      score = model(ip_ids, attention_mask=masks)
      pos_prob.append(score.cpu().detach().numpy())
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  org_ans.append(row.fact_2)

  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_2:
      contains_fact2+=1
  # if(index==1000):
  #   break
  print(index)
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
print("QASC DEV experiment 4 with considering only fact2")
print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_fact2",contains_fact2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# **BM25 Baseline Model Result Evaluation**



In [ ]:
def get_candidate_passages(query):

  candidate_passages=[]
  scores=[]
  hits = searcher.search(query, k=50)
  # return the first top 10 hits:
  for hit in hits:
    doc = searcher.doc(str(hit.docid))
    candidate_passages.append(doc.raw().replace('"', ''))
    scores.append(hit.score)
  
  return candidate_passages,scores

## **Experiment 1: only  combined fact as exact answer**






In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  print(index)
  answers,score= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(list(zip(answers,score)), columns=['answer','score'])
  temp_df['question']=row.question
  # dataloader= create_dataloader(tokenizer, temp_df)
  # pos_prob= []
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  org_ans.append(row.combined_fact)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.combined_fact:
      contains_combined_fact+=1
  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("combined fact:",contains_combined_fact)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## **Experiment 2: Using all three facts as exact answer**





In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_all_three=0
contains12=0
contains1c=0
contains2c=0
for index, row in data_df.iterrows():
  print(index)
  answers,score= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(list(zip(answers,score)), columns=['answer','score'])
  temp_df['question']=row.question
  # dataloader= create_dataloader(tokenizer, temp_df)
  # pos_prob= []
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  org_ans.append(row.combined_fact)
  org_ans.append(row.fact_1)
  org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)):
    contains_all_three+=1
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact not in top_k_ans)):
    contains12+=1
  if ((row.fact_1 in top_k_ans) and (row.combined_fact in top_k_ans) and (row.fact_2 not in top_k_ans)):
    contains1c+=1
  if ((row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)and (row.fact_1 not in top_k_ans)):
    contains2c+=1

  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
  

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_all_three",contains_all_three)
print("contains_fact1_and_fact2",contains12)
print("contains_fact1_and_factc",contains1c)
print("contains_fact2_and_factc",contains2c)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## **Experiment 3: only  fact1 as exact answer**






In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact1=0

for index, row in data_df.iterrows():
  print(index)
  answers,score= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(list(zip(answers,score)), columns=['answer','score'])
  temp_df['question']=row.question
  # dataloader= create_dataloader(tokenizer, temp_df)
  # pos_prob= []
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  # org_ans.append(row.combined_fact)
  org_ans.append(row.fact_1)
  # org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_1:
      contains_fact1+=1
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_fact1",contains_fact1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## **Experiment 4: only  fact2 as exact answer**






In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact2=0

for index, row in data_df.iterrows():
  print(index)
  answers,score= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(list(zip(answers,score)), columns=['answer','score'])
  temp_df['question']=row.question
  # dataloader= create_dataloader(tokenizer, temp_df)
  # pos_prob= []
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  # org_ans.append(row.combined_fact)
  org_ans.append(row.fact_1)
  # org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_2:
      contains_fact2+=1
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_fact1",contains_fact2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### **Classification Baseline Model experimentation**



# Loading the model

In [ ]:
#classification model

model_path= '/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/phase2_experiments/CLS_EXPERIMENT/roberta_cls_model/'
model= RobertaForSequenceClassification.from_pretrained(model_path)
lr_weights= torch.load(join(model_path, 'model_state.bin'))
#model.out.load_state_dict(lr_weights)
model.to(device)



tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Some weights of the model checkpoint at /content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/sia_experiment/roberta_cls_model/ were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/sia_experiment/roberta_cls_model/ and are newly initialized: ['classifier.out_proj.weight', 'classifier.den

In [ ]:
data_df=pd.read_csv('/content/drive/My Drive/phase2_experiments/Data/dev_facts.csv')

In [ ]:
data_df

,question,fact_1,fact_2,combined_fact
0,Climate is generally described in terms of what?,Climate is generally described in terms of tem...,Fire behavior is driven by local weather condi...,Climate is generally described in terms of loc...
1,what varies by altitude?,Climate is generally described in terms of tem...,Instructions Climate Climate varies according ...,temperature and moisture varies by alltitude
2,What is changing globally?,Climate is generally described in terms of tem...,Climate change is a global problem.,temperature and moisture is changing globally
3,what are generally described in terms of tempe...,Climate is generally described in terms of tem...,Introduction to Climate Introduction Climate i...,average weather conditions are generally descr...
4,What is described in terms of temperature and ...,Climate is generally described in terms of tem...,Clouds are made of moisture and the moisture i...,Climate is usually described in terms of tempe...
...,...,...,...,...
921,Kidney failure may be treated by the mechanica...,Kidney failure may be treated with dialysis.,Dialysis is the mechanical process of cleaning...,Kidney failure may be treated by the mechanica...
922,What can dialysis be used to treat?,Kidney failure may be treated with dialysis.,"Diabetic kidney disease, kidney failure and ot...",One effect of diabetes can be treated with dia...
923,What are the consequences of not getting dialy...,Kidney failure may be treated with dialysis.,Kidney failure is a death sentence.,A lack of dialysis may lead to death.
924,Kidney failure may be treated by cleaning the ...,Kidney failure may be treated with dialysis.,Dialysis Dialysis is a way of cleaning the blo...,Kidney failure may be treated by cleaning the ...


## **Experiment 1: only combined fact as exact answer**


In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  org_ans.append(row.combined_fact)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.combined_fact:
      contains_combined_fact+=1
  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("combined fact:",contains_combined_fact)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## **Experiment 2: All three facts as exact answer**


In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_all_three=0
contains12=0
contains1c=0
contains2c=0
for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  org_ans.append(row.combined_fact)
  org_ans.append(row.fact_1)
  org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)):
    contains_all_three+=1
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact not in top_k_ans)):
    contains12+=1
  if ((row.fact_1 in top_k_ans) and (row.combined_fact in top_k_ans) and (row.fact_2 not in top_k_ans)):
    contains1c+=1
  if ((row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)and (row.fact_1 not in top_k_ans)):
    contains2c+=1

  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
  

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_all_three",contains_all_three)
print("contains_fact1_and_fact2",contains12)
print("contains_fact1_and_factc",contains1c)
print("contains_fact2_and_factc",contains2c)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## **Experiment 3: only Fact1 as exact answer**


In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact1=0

for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  # org_ans.append(row.combined_fact)
  org_ans.append(row.fact_1)
  # org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_1:
      contains_fact1+=1
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_fact1",contains_fact1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## **Experiment 4: only Fact2 as exact answer**



In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact2=0

for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  # org_ans.append(row.combined_fact)
  # org_ans.append(row.fact_1)
  org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_2:
      contains_fact2+=1
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_fact2_and_factc",contains_fact2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27